# Fashion MNIST

In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices())

2023-09-27 12:19:54.068477: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-09-27 12:19:55.491828: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-27 12:19:55.496762: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-27 12:19:55.497028: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [6]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

4422102/4422102 [==============================] - 0s 0us/step


In [10]:
print('train_images:', train_images.shape)
print('test_images:', test_images.shape)
print('train_labels:', train_labels)
print('test_labels:', test_labels)

train_images: (60000, 28, 28)
test_images: (10000, 28, 28)
train_labels: [9 0 0 ... 3 0 5]
test_labels: [9 2 1 ... 8 1 5]


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
EPOCH_NUM = 800
VAL_RATIO = 0.2
VERBOSE_LVL = 0

from tensorflow.keras.optimizers import SGD, Adam
OPTIMIZER = lambda: SGD(learning_rate=0.01, momentum=0.01)
# OPTIMIZER = lambda: Adam()

## CVNN

In [ ]:
import cvnn.layers as complex_layers
from cvnn.losses import ComplexMeanSquareError

def make_cvnn_model(verbose = False):
    model = tf.keras.models.Sequential()
    model.add(complex_layers.ComplexInput(input_shape=(1,)))
    model.add(complex_layers.ComplexDense(units=32, activation='cart_relu'))
    model.add(complex_layers.ComplexDense(units=1, activation='linear'))
    model.compile(loss=ComplexMeanSquareError(), optimizer=OPTIMIZER())
    if verbose:
        model.summary()
    return model

cvnn_model = make_cvnn_model(verbose=True)

### Liczba parametrów w CVNN
input: 1    -> 0

dense: 32   -> 2 * (32 * (1 + 1)) = 128

dense: 1    -> 2 * (1 * (32 + 1)) = 66

            sum: 128 + 66  = 194

In [ ]:
cvnn_history = cvnn_model.fit(np.array(data), expected_output, epochs=EPOCH_NUM, validation_split=VAL_RATIO, verbose=VERBOSE_LVL)

In [ ]:
plot_loss_history(cvnn_history, "CVNN")

## RVNN

In [ ]:
def convert_to_cartesian(complex):
    return np.array([(np.real(z), np.imag(z)) for z in complex])
rvnn_input = convert_to_cartesian(data)
rvnn_output = convert_to_cartesian(expected_output)

In [ ]:
from tensorflow.keras.losses import MeanSquaredError

def make_rvnn_model(verbose = False):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(2,)))
    model.add(tf.keras.layers.Dense(38, activation='relu'))
    model.add(tf.keras.layers.Dense(2))
    model.compile(loss=MeanSquaredError(), optimizer=OPTIMIZER())
    if verbose:
        model.summary()
    return model

rvnn_model = make_rvnn_model(verbose=True)

### Liczba parametrów w RVNN
input: 2    -> 0

dense: 77   -> 77 * (2 + 1) = 231

dense: 2    -> 2 * (77 + 1) = 156

            sum: 231 + 156  = 387

In [ ]:
rvnn_history = rvnn_model.fit(rvnn_input, rvnn_output, epochs=EPOCH_NUM, validation_split=VAL_RATIO, verbose=VERBOSE_LVL)

In [ ]:
plot_loss_history(rvnn_history, "RVNN")

## Comparision

In [ ]:
plot_loss_history_comparision(rvnn_history, cvnn_history, 2)

In [ ]:
# learning_rate_list = [0.005, 0.01, 0.02]
# momentum_list = [0., 0.01, 0.05, 0.2]

# for learning_rate in learning_rate_list:
#     for momentum in momentum_list:
#         OPTIMIZER = lambda: SGD(jit_compile=True, learning_rate=learning_rate, momentum=momentum)
        
#         rvnn_model = make_rvnn_model()
#         rvnn_model.compile(loss=MeanSquaredError(), optimizer=OPTIMIZER())
#         rvnn_history = rvnn_model.fit(rvnn_input, rvnn_output, epochs=EPOCH_NUM, validation_split=VAL_RATIO, verbose=VERBOSE_LVL)
        
#         cvnn_model = make_cvnn_model()
#         cvnn_model.compile(loss=ComplexMeanSquareError(), optimizer=OPTIMIZER())
#         cvnn_history = cvnn_model.fit(np.array(data), expected_output, epochs=EPOCH_NUM, validation_split=VAL_RATIO, verbose=VERBOSE_LVL)
        
#         print(f'macro parameters: learning_rate={learning_rate}, momentum={momentum}')
#         plot_loss_history_comparision(rvnn_history, cvnn_history, 2)